In [2]:
#CRD Fund Check
#Developed by: ASkotnicki
#Version: 1.0
#Created: 18/01/2021
#Last Edited: 01/03/2021
#Last edit: Updated the input files to Feb / Jan 2021

### Develop program for checking CRD holdings for fund changes

Program should include:
- option to import CRD files
- Functionality to detect new funds
- Functionality to detect old funds
- Functionality to detect interfunding changes that are due to either; new funds, terminated funds, or just interfunding changes
- Resulting fund list should be divided as above
- Should also include a mapping to fund names
- Output in Excel with a section for notes

### Import relevant libraries

In [1]:
import numpy as np
import pandas as pd

### Create classes

In [2]:
class CompareCRD:
    
    """Object to compare the changes in fund lists between two sets of Charles River Holdings Data"""
    
    def __init__(self, current_excel, current_sheet, previous_excel, previous_sheet):
        self.current_excel = current_excel
        self.current_sheet = current_sheet
        self.current = self._current_df()
        
        self.previous_excel = previous_excel
        self.previous_sheet = previous_sheet
        self.previous = self._previous_df()
        
        self.current_units = self._current_units()
        self.previous_units = self._previous_units()
        
    def _current_df(self):
        return pd.read_excel(self.current_excel, self.current_sheet)
    
    def _previous_df(self):
        return pd.read_excel(self.previous_excel, self.previous_sheet)
    
    def _current_units(self):
        units = self.current[(self.current.SEC_TYP_CD=='UNIT') | (self.current.SEC_TYP_CD == 'UNITA')]
        units['interfunding_code'] = units.ACCT_CD.astype(str) + ' ' + units.EXT_SEC_ID.astype(str)
        units = units[['ACCT_CD','EXT_SEC_ID','interfunding_code']]
        units.columns = ['child','parent','interfunding_code']
        units['parent'] = units.child.str.replace(r'UT$','')
        units['interfunding_code'] = units.interfunding_code.str.replace(r'UT$','')
        
        return units
    
    def _previous_units(self):
        units = self.previous[(self.previous.SEC_TYP_CD=='UNIT') | (self.previous.SEC_TYP_CD == 'UNITA')]
        units['interfunding_code'] = units.ACCT_CD.astype(str) + ' ' + units.EXT_SEC_ID.astype(str)
        units = units[['ACCT_CD','EXT_SEC_ID','interfunding_code']]
        units.columns = ['child','parent','interfunding_code']
        units['parent'] = units.child.str.replace(r'UT$','')
        units['interfunding_code'] = units.interfunding_code.str.replace(r'UT$','')
        
        return units
    
    def new_funds(self):
        new_funds = []
        
        for i in self.current['ACCT_CD'].unique() :
            if i not in self.previous['ACCT_CD'].unique() :
                new_funds.append(i)
                
        return new_funds
    
    def terminated_funds(self):
        terminated_funds = []
        
        for i in self.previous['ACCT_CD'].unique() :
            if i not in self.current['ACCT_CD'].unique() :
                terminated_funds.append(i)
                
        return terminated_funds
    
    def total_new_interfund_rels(self):
        new_interfunds = []
        
        for i in self.current_units['interfunding_code'].unique():
            if i not in self.previous_units['interfunding_code'].unique():
                new_interfunds.append(i)
                
        return new_interfunds 
    
    def total_terminated_interfund_rels(self):
        terminated_interfunds = []
        
        for i in self.previous_units['interfunding_code'].unique():
            if i not in self.current_units['interfunding_code'].unique():
                terminated_interfunds.append(i)
                
        return terminated_interfunds
    
    def new_fund_interfunds(self):
        """This function returns a list of new interfunding relationships that have occurred because of the creation 
        of a new parent fund"""
        new_fund_interfund = []
        
        input_one = self.total_new_interfund_rels()
        input_two = self.new_funds()
        
        output = [i for e in input_two for i in input_one if e in i]
        
        return output
    
    def terminated_fund_interfunds(self):
        """This function returns a list of terminated interfunding relationships that have occurred because of the
        termination of the old parent fund"""
        terminated_fund_interfund = []
        
        input_one = self.total_terminated_interfund_rels()
        input_two = self.terminated_funds()
        
        output = [i for e in input_two for i in input_one if e in i]
        
        return output
    
    def other_new_interfunds(self):
        """This function returns a list of new interfunding relationships that are not explaied by new parent fund
        creation"""
        input_one = self.total_new_interfund_rels()
        input_two = self.new_fund_interfunds()
        
        output = [b for b in input_one if
          all(a not in b for a in input_two)]
        
        return output
    
    def other_term_interfunds(self):
        """This function returns a list of old interfunding relationships that are not explained by parent fund
        termination"""
        input_one = self.total_terminated_interfund_rels()
        input_two = self.terminated_fund_interfunds()
        
        output = [b for b in input_one if
          all(a not in b for a in input_two)]
        
        return output
    
    def summary_new_funds(self):
        """This function returns a dataframe of new funds created along with a blank comments section"""
        df = pd.DataFrame(self.new_funds(),columns=['fund_code'])
        df['comments'] = ''
        df['change_management_required'] = ''
        df.set_index('fund_code',inplace=True)
        
        return df
    
    def summary_terminated_funds(self):
        """This function returns a dataframe of new funds created along with a blank comments section"""
        df = pd.DataFrame(self.terminated_funds(),columns=['fund_code'])
        df['comments'] = ''
        df['change_management_required'] = ''
        df.set_index('fund_code',inplace=True)
        
        return df
    
    def summary_interfunding(self):
        """This function returns a dataframe with interfunding relationship changes and blank comments section"""
        df_new = pd.DataFrame(self.new_fund_interfunds(),columns=['interfunding_code'])
        df_new[['child','parent']] = df_new.interfunding_code.str.split(' ', n=1, expand=True)
        df_new['reason'] = 'new fund creation'
        
        df_new_oth = pd.DataFrame(self.other_new_interfunds(),columns=['interfunding_code'])
        df_new_oth[['child','parent']] = df_new_oth.interfunding_code.str.split(' ', n=1, expand=True)
        df_new_oth['reason'] = 'new interfunding relationship'
        
        df_term = pd.DataFrame(self.terminated_fund_interfunds(),columns=['interfunding_code'])
        df_term[['child','parent']] = df_term.interfunding_code.str.split(' ', n=1, expand=True)
        df_term['reason'] = 'old fund terminated'
        
        df_term_oth = pd.DataFrame(self.other_term_interfunds(),columns=['interfunding_code'])
        df_term_oth[['child','parent']] = df_term_oth.interfunding_code.str.split(' ', n=1, expand=True)
        df_term_oth['reason'] = 'old interfunding relationship'
        
        df_all = pd.concat([df_new, df_new_oth, df_term, df_term_oth])
        df_all.set_index('interfunding_code',inplace=True)
        df_all['comments'] = ''
        df_all['change_management_required'] = ''
        
        return df_all

In [3]:
fund_codes = pd.read_excel(r'fundcodes.xlsx','fundcodes')

In [4]:
fund_codes.head()

,fund_code,fund_name
0,2BGF,WESTPAC VARIABLE INCOME PLAN - BALANCED GROWTH...
1,2CGP,WESTPAC VARIABLE INCOME PLAN - CASH GUARANTEED...
2,2GPP,WESTPAC VARIABLE INCOME PLAN - GROWTH PLUS (PUPS)
3,2MGF,WESTPAC VARIABLE INCOME PLAN - MODERATE GROWTH...
4,2MGP,WESTPAC VARIABLE INCOME PLAN - MANAGED GROWTH ...


In [5]:
fund_code_dict = dict(zip(fund_codes.fund_code, fund_codes.fund_name))

In [6]:
fund_code_dict

{'2BGF': 'WESTPAC VARIABLE INCOME PLAN - BALANCED GROWTH (PUPS)',
 '2CGP': 'WESTPAC VARIABLE INCOME PLAN - CASH GUARANTEED (PUPS)',
 '2GPP': 'WESTPAC VARIABLE INCOME PLAN - GROWTH PLUS (PUPS)',
 '2MGF': 'WESTPAC VARIABLE INCOME PLAN - MODERATE GROWTH (PUPS)',
 '2MGP': 'WESTPAC VARIABLE INCOME PLAN - MANAGED GROWTH (PUPS)',
 '3BGF': 'WESTPAC FOUNDATION PLAN - BALANCED GROWTH (PUPS)',
 '3CBP': 'WESTPAC INVESTMENT & PROTECTION PLAN - CASH BASED (PUPS)',
 '3CPP': 'WESTPAC FOUNDATION PLAN - CASH PLUS PORTFOLIO (PUPS)',
 '3DGF': 'WESTPAC UNIVERSAL LIFE -  DYNAMIC GROWTH (PUPS)',
 '3DGP': 'WESTPAC  INSURANCE  BOND - DYNAMIC GROWTH (PUPS)',
 '3DIG': 'WESTPAC FOUNDATION PLAN - DIVERSIFIED GROWTH (PUPS)',
 '3DSP': 'WESTPAC FOUNDATION PLAN - DIVERSIFIED STABLE (PUPS)',
 '3GAF': 'WESTPAC UNIVERSAL LIFE -  GROWTH ASSETS (PUPS)',
 '3GAP': 'WESTPAC  INSURANCE  BOND - GROWTH ASSETS (PUPS)',
 '3GPP': 'WESTPAC INVESTMENT & PROTECTION PLAN - GROWTH PLUS (PUPS)',
 '3IAF': 'WESTPAC UNIVERSAL LIFE -  INCOME

# Testing

#### Current holdings

In [7]:
input_current = r'G:\IM Distribution Client Service\Research Reporting\1. SOURCE DATA\INTERNAL\CHARLES RIVER - SQL\CRTS210228.xlsx'
sheet_current = 'Sheet1'
input_previous = r'G:\IM Distribution Client Service\Research Reporting\1. SOURCE DATA\INTERNAL\CHARLES RIVER - SQL\CRTS210131.xlsx'
sheet_previous = 'Sheet1'

In [8]:
compare = CompareCRD(input_current, sheet_current, input_previous, sheet_previous)

<ipython-input-2-dc76a3412178>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  units['interfunding_code'] = units.ACCT_CD.astype(str) + ' ' + units.EXT_SEC_ID.astype(str)
<ipython-input-2-dc76a3412178>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  units['interfunding_code'] = units.ACCT_CD.astype(str) + ' ' + units.EXT_SEC_ID.astype(str)


#### Create summary for new funds

In [9]:
interfunding_results = compare.summary_interfunding()

ValueError: Columns must be same length as key

In [10]:
interfunding_results['child_name'] = interfunding_results.child.map(fund_code_dict)
interfunding_results['parent_name'] = interfunding_results.parent.map(fund_code_dict)

NameError: name 'interfunding_results' is not defined

In [15]:
terminated_results = compare.summary_terminated_funds()

In [17]:
terminated_results.head()

,comments,change_management_required,fund_name
fund_code,,,
2BGF,terminated fund,,WESTPAC VARIABLE INCOME PLAN - BALANCED GROWTH...
2CGF,terminated fund,,NaN
2CGP,terminated fund,,WESTPAC VARIABLE INCOME PLAN - CASH GUARANTEED...
2GPP,terminated fund,,WESTPAC VARIABLE INCOME PLAN - GROWTH PLUS (PUPS)
2MGF,terminated fund,,WESTPAC VARIABLE INCOME PLAN - MODERATE GROWTH...


In [19]:
terminated_results['comments'] = 'terminated fund'
terminated_results['fund_name'] = terminated_results.index.map(fund_code_dict)


In [20]:
terminated_results.head()

,comments,change_management_required,fund_name
fund_code,,,
2BGF,terminated fund,,WESTPAC VARIABLE INCOME PLAN - BALANCED GROWTH...
2CGF,terminated fund,,NaN
2CGP,terminated fund,,WESTPAC VARIABLE INCOME PLAN - CASH GUARANTEED...
2GPP,terminated fund,,WESTPAC VARIABLE INCOME PLAN - GROWTH PLUS (PUPS)
2MGF,terminated fund,,WESTPAC VARIABLE INCOME PLAN - MODERATE GROWTH...


In [21]:
new_results = compare.summary_new_funds()

In [22]:
new_results['comments'] = 'new fund created'
new_results['fund_name'] = new_results.index.map(fund_code_dict)

In [23]:
new_results.head()

,comments,change_management_required,fund_name
fund_code,,,
BTPAF3X_ALVS,new fund created,,NaN
BTPAF3X_FI,new fund created,,NaN
BTPAF3X_MACRO,new fund created,,NaN


In [37]:
all_results = pd.concat([new_results, terminated_results, interfunding_results])

In [38]:
all_results.head()

,comments,change_management_required,fund_name,child,parent,reason,child_name,parent_name
SFLRCON,new fund created,,BT SUPER FOR LIFE - CONSERVATIVE FUND - RETIRE...,NaN,NaN,NaN,NaN,NaN
SFLRGRO,new fund created,,BT SUPER FOR LIFE - GROWTH FUND - RETIREMENT,NaN,NaN,NaN,NaN,NaN
SFLRMOD,new fund created,,BT SUPER FOR LIFE - MODERATE FUND - RETIREMENT,NaN,NaN,NaN,NaN,NaN
SFLSCON,new fund created,,BT SUPER FOR LIFE - CONSERVATIVE FUND - SAVINGS,NaN,NaN,NaN,NaN,NaN
SFLSGRO,new fund created,,BT SUPER FOR LIFE - GROWTH FUND - SAVINGS,NaN,NaN,NaN,NaN,NaN


In [39]:
all_results.tail()

,comments,change_management_required,fund_name,child,parent,reason,child_name,parent_name
MFTP BTHGABF,,,NaN,MFTP,BTHGABF,old interfunding relationship,Pendal Balanced Returns Fund,NaN
MFTP BTDYEQ,,,NaN,MFTP,BTDYEQ,old interfunding relationship,Pendal Balanced Returns Fund,Pendal Dynamic Global Equity Fund
RAMWT WSP-IF,,,NaN,RAMWT,WSP-IF,old interfunding relationship,Pendal Active Balanced Fund,Pendal Enhanced Global Fixed Interest Fund
RAMWT JOHEMO,,,NaN,RAMWT,JOHEMO,old interfunding relationship,Pendal Active Balanced Fund,JOHCM Global Emerging Markets Opportunities
RAMWT BTDYEQ,,,NaN,RAMWT,BTDYEQ,old interfunding relationship,Pendal Active Balanced Fund,Pendal Dynamic Global Equity Fund


# Output

### Separate Excel Sheets

In [27]:
output_excel = 'CRD Jan 21 v2.xlsx'

In [28]:
with pd.ExcelWriter(output_excel, engine='openpyxl',mode='a') as writer :
    new_results.to_excel(writer,sheet_name='new funds')
    terminated_results.to_excel(writer, sheet_name='terminated funds')
    interfunding_results.to_excel(writer, sheet_name='interfunding changes')